<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/index.html)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 3 Assignment: Simple Classification Neural Network**

**Student Name: Your Name**

# Assignment Instructions

For this assignment, you will use the **crx** dataset. You can find the CSV file on my data site, at this location: [crx](https://data.heatonresearch.com/data/t81-558/crx.csv). Load and summarize the data set.  You will submit this summarized dataset to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

The RAW datafile looks something like the following:

|a1|a2|s3|a4|a5|a6|a7|a8|a9|a10|a11|a12|a13|a14|a15|a16|
|--|--|--|--|--|--|--|--|--|---|---|---|---|---|---|---|
|b|30.83|0|u|g|w|v|1.25|t|t|1|f|g|202|0|+|
|a|58.67|4.46|u|g|q|h|3.04|t|t|6|f|g|43|560|+|
|...|...|...|...|...|...|...|...|...|...|...|...|...|...|...|...|

For this assignment you must complete the following.

* Write a classification neural network that predicts the probability of either "+" or "-" for the column **a16**.
* Use early stopping to know when to complete your training.
* For all columns that are categorical, you must convert them to dummy variables.
* Some columns have missing values, fill these missing values with the median of that column.
* This is a simple neural network using basic techniques, do not worry too much about overall accuracy.
* Predict/submit for the entire dataset that I gave you, training and validation, you should have the same number of rows as crx.csv (690 data and 1 header row).

Your submit will look something like the following:

|+|-|
|-|-|
|0.09405358|0.90594643|
|0.33253232|0.66746765|
|0.098494485|0.90150553|
|...|...|

Common errors that you may run into include:

* **ValueError: could not convert string to float: ...** - Value errors typically mean you've not converted all of the categoricals to dummy variables.

* **tloss nan:** - Nan's usually mean youve not filled all missing values.



# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Early stopping (see module 3.4)
import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch

device = (
    "mps"
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Mounted at /content/drive
Note: using Google CoLab
Using device: cpu


<ipython-input-2-aada6cd16a09>:47: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  if getattr(torch, "has_mps", False)


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,course,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'course':course, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #3 Sample Code

The following code provides a starting point for this assignment.

In [ ]:
import os
import pandas as pd
from scipy.stats import zscore
import numpy as np
import torch
import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset


# This is your student key that I emailed to you at the beginnning of the semester.
# key = "Gx5en9cEVvaZnjut6vfLm1HG4ZO4PsI32sgldAXj"  # This is an example key and will not work.
key = ""

# You must also identify your source file.  (modify for your local setup)
# file='/content/drive/My Drive/Colab Notebooks/c_assignment_yourname_class3.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class3.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class3.ipynb'  # Mac/Linux

file= '/content/drive/My Drive/Colab Notebooks/assignment_ZhijiangLi_class3.ipynb'


# Begin assignment

df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/crx.csv",na_values=['?'])

# Your code goes here.

df.fillna(df.median(numeric_only=True), inplace=True)

for i in df.select_dtypes(include=np.number).columns:
    median = df[i].median()
    df[i].fillna(median, inplace=True)

for i in df.select_dtypes(include=['object']).columns:
    if df[i].isnull().any():
        mode_val = df[i].mode()[0]
        df[i].fillna(mode_val, inplace=True)


X = df.drop('a16', axis=1)
y = df['a16'].apply(lambda x: 1 if x == '+' else 0)

X = pd.get_dummies(X, drop_first=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train = torch.tensor(X_train.astype(np.float32))
X_val = torch.tensor(X_val.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32))
y_val = torch.tensor(y_val.values.astype(np.float32))

train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

class ClassifierModule(nn.Module):
    def __init__(self):
        super(ClassifierModule, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 32) # 64
        self.layer2 = nn.Linear(32, 16)
        self.layer3 = nn.Linear(16, 1)
        # self.layer4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        # x = self.relu(self.layer3(x))
        x = self.sigmoid(self.layer3(x))
        return x

model = ClassifierModule()

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
early_stopping = EarlyStopping(patience=10, min_delta=0.01, restore_best_weights=True)

#
epochs = 100

for epoch in range(epochs):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()

        output = model(data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()

    #
    model.eval()
    val_loss = 0
    with torch.no_grad():
        # val_losses = []
        for data, target in val_loader:
            output = model(data).squeeze()
            loss = criterion(output, target)
            #val_losses.append(loss.item())
            val_loss += loss.item()
        #val_loss = sum(val_losses) / len(val_losses)
        #print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss}')
    val_loss /= len(val_loader)

    # Early stopping check
    if early_stopping(model, val_loss):
        print(early_stopping.status)
        break
    else:
        print(early_stopping.status)

    print(f'Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.4f}')

#
model.eval()
with torch.no_grad():
    predictions = model(X_val).squeeze()
    predictions = (predictions > 0.5).float()
    accuracy = accuracy_score(y_val.numpy(), predictions.numpy())
    print(f'Validation Accuracy: {accuracy}')

X_all = torch.tensor(scaler.transform(X.astype(np.float32)))

model.eval()
with torch.no_grad():
    probabilities = model(X_all).squeeze()

df_submit = pd.DataFrame({
    '+': probabilities.numpy(),
    '-': 1 - probabilities.numpy()
})

# Submit it
# submit(source_file=file,data=[df_submit],key=key,course='t81-558',no=3)
submit(source_file=file,data=[df_submit],key=key,course='t81-558',no=3)



Epoch 1/100, Validation Loss: 0.6866
Improvement found, counter reset to 0
Epoch 2/100, Validation Loss: 0.6739
Improvement found, counter reset to 0
Epoch 3/100, Validation Loss: 0.6611
Improvement found, counter reset to 0
Epoch 4/100, Validation Loss: 0.6459
Improvement found, counter reset to 0
Epoch 5/100, Validation Loss: 0.6273
Improvement found, counter reset to 0
Epoch 6/100, Validation Loss: 0.6043
Improvement found, counter reset to 0
Epoch 7/100, Validation Loss: 0.5805
Improvement found, counter reset to 0
Epoch 8/100, Validation Loss: 0.5531
Improvement found, counter reset to 0
Epoch 9/100, Validation Loss: 0.5271
Improvement found, counter reset to 0
Epoch 10/100, Validation Loss: 0.5020
Improvement found, counter reset to 0
Epoch 11/100, Validation Loss: 0.4809
Improvement found, counter reset to 0
Epoch 12/100, Validation Loss: 0.4603
Improvement found, counter reset to 0
Epoch 13/100, Validation Loss: 0.4453
No improvement in the last 1 epochs
Epoch 14/100, Validati